In [1]:
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install wordcloud
!pip install langdetect
!pip install sumy

In [2]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('vader_lexicon')
import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MADHAVI\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
consumerKey = "JhRYBT3ko20lGlnMK43sKquaj"
consumerSecret = "xxnqBCcXEnIvu3aZ5ZFbKHEPsXEhEowZFeN7DDOljgmJaPAOdR"
accessToken = "1601083881431519234-7Gb8kIkAZC9pMoEf7zqdqIJAZ4Ipg6"
accessTokenSecret = "m5AYmu0wCs6A0FWsd9JuST08ADC6pu1DvEa6ovj19uLCX"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [4]:
def percentage(part,whole):
    return 100 * float(part)/float(whole) 

keyword = "ews reservation"
noOfTweet = 5000


tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)
positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets:
    tweet_list.append(tweet.text)

tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing unnecessary words, urls, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)
Tweets = tw_list["text"].values.tolist()

stopword = nltk.corpus.stopwords.words('english')
def clean_text(text):
    clean = []
    for elem in text:
        elem = elem.split()
        text_rc = " ".join([word.lower() for word in elem if word not in stopword])
        clean.append(text_rc)
    return clean

tweets = list(set(clean_text(Tweets))) #cleaned tweets are stored in tweets

#for elem in tweets:
   # print(elem)



for tweet in tweets:
    tweet_list.append(tweet)
    analysis = TextBlob(tweet)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet)
        neutral += 1
positive = percentage(positive, len(tweets))
negative = percentage(negative, len(tweets))
neutral = percentage(neutral, len(tweets))
polarity = percentage(polarity, len(tweets))
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')


#summarisation
def summarize(text):
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        from sumy.summarizers.text_rank import TextRankSummarizer
        summarizer = TextRankSummarizer()
        summary = summarizer(parser.document, 2)
        text_summary = ""
        for sentence in summary:
            text_summary += str(sentence)
        return text_summary
    
print("positive",summarize(positive_list)) 
print("negative",summarize(negative_list))
print("neutral",summarize(neutral_list)) 
print("positive percentage = ", positive)
print("negative percentage = ", negative)
print("neutral percentage = ", neutral)






positive ['india backward classes federation challenges supreme court order ews reservation', 'please ask eradicate economy ceiling correct per indian cons', 'wrong caste word mouth identity natural identity like', 'funny part brahmins baniyas talking reservat', 'akashkj supreme court ne bola hai bhai becuz supreme court mistake phela cent', 'obcreservations 27 dist ews 4 statewide reservation 3rd amp 4th grades dist ha', 'dalits lives improved much ambedkar might hoped shashi tharoor tells thefederal imple', 'one thing certain caste sole cause casteism reservation exists', 'congress gj loss hp victory hoping least congress speak ews reservation mismanagement congress', 'dear modi govt approve age amp attempt relaxation economically weaker sections ews new 10 reservation system amp', 'bela trivedi bench revoked judgement ordering release gn saibaba also ews', 'happy ews reservation', 'psychic spiritual intellect sc st students tends lack bahujansamaj india jaibhim instance h', 'underst

In [5]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  2925
positive number:  231
negative number:  126
neutral number:  369


In [6]:
#Calculating Negative, Positive, Neutral and Compound values

tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

tw_list.head(10)


,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
1,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
2,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
3,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
4,ನಿಂತು ಹೊಯ್ಕೋಳಾದ್ ಒಂದೇ ಬಾಕಿ ಆದ 🤧 ಹೊಯ್ಕೋರಿ ಜಲ್ದಿ...,...,0.0,0.0,negative,0.375,0.625,0.000,-0.0516
5,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
6,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
7,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
8,@srihari923 @siddaramaiah @BJP4Karnataka @BJP4...,yes please remove ews reservation,0.0,0.0,positive,0.000,0.375,0.625,0.6124
9,RT @DravidianForum: Join us on Twitter Spaces ...,join us on twitter spaces today at 6 pm ist w...,0.0,0.0,positive,0.000,0.891,0.109,0.2960
